Nhóm 18
- Đỗ Tấn Lực - 23520903
- Lê Quang Long - -23520878

#### Import các thư viện cần thiết cho xử lý dữ liệu, xây dựng mô hình, và Spark

- pandas, numpy: Xử lý dữ liệu.
- torch, torch.nn, torch.optim: Xây dựng và huấn luyện mô hình deep learning.
- sklearn: Tiền xử lý, chia tập, đánh giá mô hình.
- re: Xử lý chuỗi, regex.
- kagglehub: Tải dataset từ Kaggle.
- pyspark: Xử lý dữ liệu lớn với Spark, định nghĩa UDF.
- transformers: Sử dụng mô hình BERT (PhoBERT).

In [2]:
%pip install pyspark

     ---------------------------------------- 0.0/434.2 MB ? eta -:--:--
     ---------------------------------------- 0.8/434.2 MB 6.7 MB/s eta 0:01:05
     ---------------------------------------- 1.8/434.2 MB 5.6 MB/s eta 0:01:18
     ---------------------------------------- 3.1/434.2 MB 7.4 MB/s eta 0:00:59
     ---------------------------------------- 3.1/434.2 MB 7.4 MB/s eta 0:00:59
     ---------------------------------------- 3.4/434.2 MB 3.8 MB/s eta 0:01:54
     ---------------------------------------- 5.2/434.2 MB 4.5 MB/s eta 0:01:36
      --------------------------------------- 5.8/434.2 MB 4.2 MB/s eta 0:01:43
      --------------------------------------- 6.6/434.2 MB 4.2 MB/s eta 0:01:44
      --------------------------------------- 7.9/434.2 MB 4.4 MB/s eta 0:01:37
      --------------------------------------- 9.2/434.2 MB 4.6 MB/s eta 0:01:32
      -------------------------------------- 10.2/434.2 MB 4.7 MB/s eta 0:01:31
      -------------------------------------- 11

In [1]:
import pandas as pd # Thư viện xử lý dữ liệu dạng bảng
import numpy as np # Thư viện xử lý mảng và tính toán số học
import torch # Thư viện PyTorch cho deep learning
import torch.nn as nn # Các lớp neural network trong PyTorch
import torch.optim as optim # Các thuật toán tối ưu hóa trong PyTorch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader # Công cụ xử lý dữ liệu theo batch trong PyTorch
from sklearn.model_selection import train_test_split # Chia dữ liệu thành tập train/test/val
from sklearn.preprocessing import LabelEncoder # Mã hóa nhãn
from sklearn.metrics import accuracy_score # Tính toán độ chính xác
from collections import Counter # Đếm tần suất các phần tử
import json
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import re # Thư viện xử lý chuỗi bằng regex
from pyspark.sql import SparkSession # Tạo và quản lý SparkSession
from pyspark.sql.types import DoubleType, ArrayType, StringType, IntegerType # Các kiểu dữ liệu trong Spark SQL
from pyspark.sql.functions import pandas_udf, PandasUDFType # Định nghĩa Pandas UDF
from pyspark.ml.functions import predict_batch_udf # Định nghĩa batch UDF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator # Đánh giá mô hình phân loại đa lớp
from transformers import AutoTokenizer, AutoModel # Tải tokenizer và model từ Hugging Face Transformers

#### Tải dataset ABSA tiếng Việt từ Kaggle

- Sử dụng kagglehub để tải về bộ dữ liệu.
- Đọc các file train, test, val bằng pandas.

In [4]:
# Đọc các file CSV vào DataFrame bằng pandas
data = pd.read_excel("test_flow_reviews_1_labeled_full.xlsx")
train, test = train_test_split(data, test_size=0.2, random_state=42)

FileNotFoundError: [Errno 2] No such file or directory: 'test_flow_reviews_1_labeled_full.xlsx'

## BERT + Pandas UDF

- Sử dụng mô hình BERT (PhoBERT) để dự đoán song song trên Spark DataFrame.
- Trả về dự đoán cho từng review dưới dạng chuỗi.
- Đánh giá lại độ chính xác trên Spark DataFrame.

#### Cài đặt thư viện transformers

- Sử dụng pip để cài đặt phiên bản `transformers==4.40.2`.

In [ ]:
# Cài đặt thư viện transformers phiên bản 4.40.2 (chế độ im lặng)
# !pip install transformers --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 142.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.0 MB/s eta 0:00:00


#### Tải tokenizer và mô hình PhoBERT

- Tải `AutoTokenizer` và `AutoModel` từ thư viện `transformers`.
- Khởi tạo tokenizer cho mô hình `vinai/phobert-base`.

In [6]:
from transformers import AutoTokenizer, AutoModel # Tải tokenizer và model từ Hugging Face Transformers

# Khởi tạo tokenizer cho mô hình PhoBERT-base
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

c:\Users\Luc\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Luc\.cache\huggingface\hub\models--vinai--phobert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


#### Định nghĩa mô hình ABSA dựa trên BERT và khởi tạo

- Định nghĩa lớp `BERT_ABSA` kế thừa từ `nn.Module`.
- Mô hình bao gồm lớp BERT (`vinai/phobert-base`), dropout, và lớp tuyến tính để dự đoán phân loại cho từng khía cạnh.
- Khởi tạo mô hình, hàm mất mát `CrossEntropyLoss`, và optimizer `Adam`.
- Chuyển mô hình sang thiết bị (CPU/GPU) được định nghĩa bởi biến `DEVICE`.

In [7]:
ASPECTS = ['chat_luong_san_pham', 'gia_ca', 'van_chuyen_giao_hang', 'dong_goi_bao_bi', 'dich_vu_cskh', 'mo_ta_hinh_anh', 'loi_bao_hanh_hang_gia', 'trai_nghiem_su_dung', 'uy_tin_thai_do_shop', 'khuyen_mai_voucher'] # Danh sách các khía cạnh (aspects)
NUM_CLASSES = 4 # Số lượng lớp (sentiment: -1, 0, 1, 2)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Chọn thiết bị (GPU hoặc CPU) để chạy mô hình
MAX_LEN = 128

label_map = {-1: 0, 0: 1, 1: 2, 2: 3} # Ánh xạ các nhãn gốc (-1, 0, 1, 2) sang chỉ số (0, 1, 2, 3)

# Lấy các cột nhãn từ DataFrame train và chuyển đổi sang mảng numpy
train_labels_raw = np.array(train[ASPECTS])
# Áp dụng ánh xạ label_map cho từng giá trị trong mảng nhãn, gán giá trị mặc định là 0 nếu không tìm thấy trong map
train_labels = np.vectorize(lambda x: label_map.get(x, 0))(train_labels_raw)  # Default to 0 if unexpected

# Tương tự cho tập test
test_labels_raw = np.array(test[ASPECTS])
test_labels = np.vectorize(lambda x: label_map.get(x, 0))(test_labels_raw)

# Tương tự cho tập validation
# val_labels_raw = np.array(val[ASPECTS])
# val_labels = np.vectorize(lambda x: label_map.get(x, 0))(val_labels_raw)

In [8]:
# Định nghĩa lớp mô hình BERT cho bài toán ABSA
class BERT_ABSA(nn.Module):
    # Khởi tạo mô hình với số lượng khía cạnh và số lượng lớp
    def __init__(self, num_aspects=len(ASPECTS), num_classes=NUM_CLASSES):
        super().__init__()
        # Tải mô hình BERT đã được huấn luyện trước (PhoBERT-base)
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        # Lớp Dropout để tránh overfitting
        self.dropout = nn.Dropout(0.1)
        # Lớp tuyến tính cuối cùng để dự đoán kết quả cho từng khía cạnh
        self.fc = nn.Linear(768, num_aspects * num_classes)
        self.num_classes = num_classes
        self.num_aspects = num_aspects

    # Định nghĩa phương thức forward để truyền dữ liệu qua mô hình
    def forward(self, input_ids, attention_mask):
        # Truyền input_ids và attention_mask qua mô hình BERT
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # Lấy output từ lớp pooler (thường là embedding của token [CLS])
        pooled_output = output.pooler_output
        # Áp dụng Dropout
        drop = self.dropout(pooled_output)
        # Truyền qua lớp tuyến tính cuối cùng
        out = self.fc(drop)
        # Thay đổi kích thước output để phù hợp với bài toán (batch_size, num_aspects, num_classes)
        return out.view(-1, self.num_aspects, self.num_classes)

# Khởi tạo mô hình BERT_ABSA và chuyển nó sang thiết bị đã định nghĩa (CPU/GPU)
model_bert = BERT_ABSA().to(DEVICE)
# Định nghĩa hàm mất mát Cross-Entropy
criterion = nn.CrossEntropyLoss()
# Định nghĩa bộ tối ưu hóa Adam
optimizer = optim.Adam(model_bert.parameters(), lr=2e-5)

#### Token hóa dữ liệu và chuẩn bị Tensor

- Định nghĩa hàm `tokenize` để chuyển đổi văn bản thành input_ids và attention_mask sử dụng tokenizer của PhoBERT.
- Áp dụng hàm `tokenize` cho cột 'Review' trong các tập train, test, và val.
- Nối các tensor input_ids và attention_mask lại với nhau cho từng tập dữ liệu.

In [9]:
# Định nghĩa hàm tokenize để chuyển đổi văn bản thành input_ids và attention_mask
def tokenize(text):
    # Sử dụng tokenizer của PhoBERT để mã hóa văn bản
    # max_length: độ dài tối đa của chuỗi đầu vào
    # padding='max_length': thêm padding để đạt độ dài tối đa
    # truncation=True: cắt bớt chuỗi nếu dài hơn max_length
    # return_tensors='pt': trả về kết quả dưới dạng PyTorch tensor
    return tokenizer.encode_plus(text, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='pt')

# Áp dụng hàm tokenize cho cột 'Review' của tập train, test và val
train_bert = [tokenize(text) for text in train['reviewContent']]
test_bert = [tokenize(text) for text in test['reviewContent']]
# val_bert = [tokenize(text) for text in val['Review']]

# Nối các tensor input_ids và attention_mask lại với nhau cho từng tập dữ liệu
train_input_ids = torch.cat([e['input_ids'] for e in train_bert])
train_attention_mask = torch.cat([e['attention_mask'] for e in train_bert])
# val_input_ids = torch.cat([e['input_ids'] for e in val_bert])
# val_attention_mask = torch.cat([e['attention_mask'] for e in val_bert])
test_input_ids = torch.cat([e['input_ids'] for e in test_bert])
test_attention_mask = torch.cat([e['attention_mask'] for e in test_bert])

#### Định nghĩa Dataset, DataLoader

- Định nghĩa lớp `ABSADataset_Bert` kế thừa từ `Dataset` để xử lý dữ liệu cho PyTorch.
- Tạo instance của Dataset cho tập train, test, và val.
- Tạo DataLoader cho từng tập dữ liệu để xử lý theo batch.


In [10]:
# Định nghĩa lớp Dataset tùy chỉnh cho bài toán ABSA sử dụng BERT
class ABSADataset_Bert(Dataset):
      # Hàm khởi tạo Dataset
      def __init__(self, input_ids, attention_masks, labels):
            self.input_ids = input_ids # Tensor chứa input_ids
            self.attention_masks = attention_masks # Tensor chứa attention_masks
            self.labels = torch.tensor(labels, dtype=torch.long) # Tensor chứa nhãn (chuyển sang kiểu long)

      # Phương thức trả về một mẫu dữ liệu tại chỉ mục idx
      def __getitem__(self, idx):
            return self.input_ids[idx], self.attention_masks[idx], self.labels[idx]

      # Phương thức trả về tổng số mẫu dữ liệu trong Dataset
      def __len__(self):
            return len(self.input_ids)

# Tạo instance của Dataset cho tập train, test và val
train_data = ABSADataset_Bert(train_input_ids, train_attention_mask, train_labels)
test_data = ABSADataset_Bert(test_input_ids, test_attention_mask, test_labels)
# val_data = ABSADataset_Bert(val_input_ids, val_attention_mask, val_labels)

# Tạo DataLoader cho từng tập dữ liệu để xử lý theo batch
# train_load: DataLoader cho tập huấn luyện (xáo trộn dữ liệu)
train_load = DataLoader(train_data, batch_size=32, shuffle=True)
# test_load: DataLoader cho tập kiểm tra
test_load = DataLoader(test_data, batch_size=32)
# val_load: DataLoader cho tập validation
# val_load = DataLoader(val_data, batch_size=32)

#### Huấn luyện và đánh giá mô hình BERT

- Thực hiện vòng lặp huấn luyện mô hình:
    - Đặt mô hình ở chế độ train (`model_bert.train()`).
    - Tính tổng mất mát qua từng batch.
    - Thực hiện backpropagation và cập nhật trọng số.
    - In ra mất mát sau mỗi epoch.
- Đánh giá mô hình trên tập validation sau khi huấn luyện:
    - Đặt mô hình ở chế độ eval (`model_bert.eval()`).
    - Dự đoán trên tập validation.
    - Tính toán độ chính xác trên tập validation.
    - In ra độ chính xác.
- Lưu trạng thái của mô hình đã huấn luyện vào file 'bert_absa_model.pth'.

In [11]:
epoch = 50 # Số lượng epoch để huấn luyện
# Vòng lặp huấn luyện qua các epoch
for e in range(epoch):
    # Đặt mô hình ở chế độ train
    model_bert.train()
    total_loss = 0 # Tổng mất mát trong epoch
    # Lặp qua từng batch dữ liệu trong train_load
    for input_ids, attention_mask, labels in train_load:
        # Chuyển dữ liệu sang thiết bị đã định nghĩa (CPU/GPU)
        input_ids, attention_mask, labels = input_ids.to(DEVICE), attention_mask.to(DEVICE), labels.to(DEVICE)
        # Đặt gradient về 0 trước khi thực hiện backpropagation
        optimizer.zero_grad()
        # Truyền dữ liệu qua mô hình để nhận output
        output = model_bert(input_ids, attention_mask)
        # Tính toán mất mát cho từng khía cạnh và cộng lại
        loss = sum(criterion(output[:, i, :], labels[:, i]) for i in range(len(ASPECTS)))
        # Thực hiện backpropagation để tính gradient
        loss.backward()
        # Cập nhật trọng số của mô hình
        optimizer.step()
        # Cộng dồn mất mát của batch vào tổng mất mát
        total_loss += loss.item()
    # In ra mất mát trung bình của epoch hiện tại
    print(f"Epoch {e+1}/{epoch}, Loss: {total_loss/len(train_load)}")

# ĐÁNH GIÁ mô hình trên tập validation
# Đặt mô hình ở chế độ eval (không cập nhật trọng số)
# model_bert.eval()
# val_pred = [] # Danh sách lưu trữ dự đoán trên tập validation
# val_true = [] # Danh sách lưu trữ nhãn thực tế trên tập validation
# val_loss = 0 # Mất mát trên tập validation (không sử dụng trong code này)
# # Không tính toán gradient trong quá trình đánh giá
# with torch.no_grad():
#     # Lặp qua từng batch dữ liệu trong val_load
#     for input_ids, attention_mask, labels in val_load:
#         # Chuyển dữ liệu sang thiết bị đã định nghĩa (CPU/GPU)
#         input_ids, attention_mask, labels = input_ids.to(DEVICE), attention_mask.to(DEVICE), labels.to(DEVICE)
#         # Truyền dữ liệu qua mô hình để nhận output
#         outputs = model_bert(input_ids, attention_mask)
#         # Mở rộng danh sách dự đoán và nhãn thực tế
#         val_pred.extend(torch.argmax(outputs, dim=2).cpu().numpy())
#         val_true.extend(labels.cpu().numpy())
# # Tính toán mất mát trung bình trên tập validation (không sử dụng trong code này)
# val_loss /= len(val_load)
# # Tính toán độ chính xác trên tập validation
# val_acc = accuracy_score(np.array(val_true).flatten(), np.array(val_pred).flatten())
# # In ra mất mát và độ chính xác trên tập validation
# print(f'Epoch {epoch+1}, Val Loss: {val_loss}, Val Accuracy: {val_acc}')

# # Lưu trạng thái của mô hình đã huấn luyện vào file 'bert_absa_model.pth'
torch.save(model_bert.state_dict(), 'bert_absa_model.pth')

Epoch 1/50, Loss: 10.20448924944951
Epoch 2/50, Loss: 6.909157111094548
Epoch 3/50, Loss: 5.858544844847459
Epoch 4/50, Loss: 5.455583975865291
Epoch 5/50, Loss: 5.270629350955669
Epoch 6/50, Loss: 5.260270613890428
Epoch 7/50, Loss: 5.131260266670814
Epoch 8/50, Loss: 4.972110436512874
Epoch 9/50, Loss: 4.672413404171284
Epoch 10/50, Loss: 4.404911068769602
Epoch 11/50, Loss: 4.202004185089698
Epoch 12/50, Loss: 3.931226501098046
Epoch 13/50, Loss: 3.6727208265891442
Epoch 14/50, Loss: 3.330933094024658
Epoch 15/50, Loss: 3.0653248750246487
Epoch 16/50, Loss: 2.8178448401964626
Epoch 17/50, Loss: 2.495008335663722
Epoch 18/50, Loss: 2.2656031250953674
Epoch 19/50, Loss: 2.034253555994767
Epoch 20/50, Loss: 1.8598335293623118
Epoch 21/50, Loss: 1.7001375326743493
Epoch 22/50, Loss: 1.5468725516245916
Epoch 23/50, Loss: 1.4287583896747003
Epoch 24/50, Loss: 1.338996451634627
Epoch 25/50, Loss: 1.2296805931971624
Epoch 26/50, Loss: 1.1417683477585132
Epoch 27/50, Loss: 1.0634305339593153

#### Đánh giá mô hình trên tập test

- Đặt mô hình ở chế độ eval (`model_bert.eval()`).
- Thực hiện dự đoán trên tập test.
- So sánh kết quả dự đoán với nhãn thực tế.
- Tính toán độ chính xác cuối cùng trên tập test và in ra.

In [12]:
# Đặt mô hình ở chế độ eval (không cập nhật trọng số)
model_bert.eval()
test_preds = [] # Danh sách lưu trữ dự đoán trên tập test
test_true = [] # Danh sách lưu trữ nhãn thực tế trên tập test
# Không tính toán gradient trong quá trình đánh giá
with torch.no_grad():
    # Lặp qua từng batch dữ liệu trong test_load
    for input_ids, attention_mask, labels in test_load:
        # Chuyển dữ liệu sang thiết bị đã định nghĩa (CPU/GPU)
        input_ids, attention_mask, labels = input_ids.to(DEVICE), attention_mask.to(DEVICE), labels.to(DEVICE)
        # Truyền dữ liệu qua mô hình để nhận output
        outputs = model_bert(input_ids, attention_mask)
        # Mở rộng danh sách dự đoán và nhãn thực tế
        test_preds.extend(torch.argmax(outputs, dim=2).cpu().numpy())
        test_true.extend(labels.cpu().numpy())
# Tính toán độ chính xác cuối cùng trên tập test
test_acc = accuracy_score(np.array(test_true).flatten(), np.array(test_preds).flatten())
# In ra độ chính xác cuối cùng trên tập test
print('Final Test Accuracy:', test_acc)

Final Test Accuracy: 0.9794117647058823


#### Dự đoán với Spark Pandas UDF (BERT)

- Sử dụng Spark để dự đoán song song trên tập test với mô hình đã huấn luyện.
- Trả về dự đoán cho từng review dưới dạng chuỗi.
- Đánh giá lại độ chính xác trên Spark DataFrame.

In [14]:
# Tạo SparkSession với tên ứng dụng 'ABSA-PandasUDF_BERT'
spark = SparkSession.builder.appName('ABSA-PandasUDF_BERT').getOrCreate()

# Tạo Spark DataFrame từ cột 'Review' của tập test
spark_test_df = spark.createDataFrame(test[['reviewContent']])

# Định nghĩa hàm Python sẽ được sử dụng làm Pandas UDF cho dự đoán ABSA với BERT
def predict_absa_bert(reviews):
    # Tải mô hình BERT đã được huấn luyện
    load_model = BERT_ABSA().to('cpu')
    # Tải trạng thái của mô hình đã lưu
    load_model.load_state_dict(torch.load('bert_absa_model.pth'))
    # Đặt mô hình ở chế độ eval
    load_model.eval()

    # Token hóa danh sách các reviews
    tokens = [tokenize(text) for text in reviews]
    # Nối các tensor input_ids lại với nhau
    input_ids = torch.cat([t['input_ids'] for t in tokens])
    # Nối các tensor attention_mask lại với nhau
    attention_mask = torch.cat([t['attention_mask'] for t in tokens])

    # Thực hiện dự đoán mà không tính toán gradient
    with torch.no_grad():
        # Truyền dữ liệu qua mô hình để nhận output
        outputs = load_model(input_ids, attention_mask)
        probs = F.softmax(outputs, dim=-1)
        probs_np = probs.cpu().numpy()
        # Lấy chỉ mục của lớp có xác suất cao nhất cho mỗi khía cạnh
        # predictions = torch.argmax(outputs, dim=2).numpy()

    # Trả về kết quả dự đoán dưới dạng Pandas Series, mỗi phần tử là chuỗi các dự đoán phân tách bằng dấu phẩy
    # return pd.Series([','.join(map(str, p)) for p in predictions])
    results = []
    for row in probs_np:
        # row có shape (Num_Aspects, 4)
        # Làm tròn 4 chữ số thập phân cho gọn nhẹ
        row_list = row.round(4).tolist()
        # Chuyển thành chuỗi JSON: "[[0.1, 0.8, 0.05, 0.05], [...], ...]"
        results.append(json.dumps(row_list))

    return pd.Series(results)

# Đăng ký hàm predict_absa_bert làm Pandas UDF trong Spark
# returnType='string': kiểu dữ liệu trả về của UDF là chuỗi
# functionType=PandasUDFType.SCALAR: loại UDF là scalar (áp dụng cho từng hàng)
absa_udf_bert = pandas_udf(predict_absa_bert, returnType='string', functionType=PandasUDFType.SCALAR)
# Áp dụng UDF vào Spark DataFrame để tạo cột 'predictions'
result_df = spark_test_df.withColumn('predictions', absa_udf_bert(spark_test_df['reviewContent']))
# Hiển thị kết quả dự đoán
result_df.show(truncate=False)

# # Tạo Spark DataFrame từ kết quả dự đoán và nhãn thực tế
# preds_df = spark.createDataFrame(pd.DataFrame({'prediction': np.array(test_preds).flatten(), 'label': np.array(test_true).flatten()}))
# # Ép kiểu cột 'prediction' sang DoubleType
# preds_df = preds_df.withColumn('prediction', preds_df['prediction'].cast(DoubleType()))
# # Ép kiểu cột 'label' sang DoubleType
# preds_df = preds_df.withColumn('label', preds_df['label'].cast(DoubleType()))

# # Khởi tạo MulticlassClassificationEvaluator để đánh giá độ chính xác
# evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
# # Đánh giá mô hình trên Spark DataFrame
# accuracy = evaluator.evaluate(preds_df)
# # In ra độ chính xác trên tập test
# print('Test Accuracy:', accuracy)

# BƯỚC 3: XỬ LÝ KẾT QUẢ ĐỂ ĐÁNH GIÁ (Phần bạn cần sửa nhất)
# ---------------------------------------------------------

preds_from_spark = result_df.select('predictions').toPandas()['predictions']
def get_label_list(row):
    return [int(label_map.get(row[col], 0)) for col in ASPECTS]

# Tạo list nhãn thật (Ground Truth) từ biến 'test'
true_labels_list = test.apply(get_label_list, axis=1).tolist()

# --------------------------------------------------------------------------------
# PHẦN 2: XỬ LÝ ĐỒNG BỘ VÀ TÍNH TOÁN
# --------------------------------------------------------------------------------

all_preds_flat = []
all_labels_flat = []

print(f"Đang đánh giá {len(preds_from_spark)} mẫu...")

# Duyệt qua từng cặp (Dự đoán từ Spark - Nhãn từ Pandas)
for pred_json, true_label in zip(preds_from_spark, true_labels_list):
    try:
        # 1. Parse JSON kết quả dự đoán (Spark trả về string JSON)
        if not pred_json: continue
        pred_probs = json.loads(pred_json) # Dạng [[0.1, ...], [...]]

        # 2. Argmax: Chuyển xác suất -> Nhãn (0,1,2,3)
        pred_labels = np.argmax(pred_probs, axis=1).tolist()

        # 3. Flatten: Gom lại để tính Accuracy tổng
        if len(pred_labels) == len(true_label):
            all_preds_flat.extend(pred_labels)
            all_labels_flat.extend(true_label)
        else:
            print(f"Lệch độ dài: Pred={len(pred_labels)} vs True={len(true_label)}")

    except Exception as e:
        print(f"Lỗi: {e}")
        continue

# --------------------------------------------------------------------------------
# PHẦN 3: TẠO DATAFRAME ĐÁNH GIÁ VÀ IN KẾT QUẢ
# --------------------------------------------------------------------------------

# Tạo Spark DataFrame mới chỉ để phục vụ Evaluator
eval_df = spark.createDataFrame(pd.DataFrame({
    'prediction': all_preds_flat,
    'label': all_labels_flat
}))

# Ép kiểu Double (bắt buộc cho Spark ML Evaluator)
eval_df = eval_df.withColumn('prediction', eval_df['prediction'].cast(DoubleType()))
eval_df = eval_df.withColumn('label', eval_df['label'].cast(DoubleType()))

# 1. Tính Accuracy
acc_eval = MulticlassClassificationEvaluator(
    labelCol='label',
    predictionCol='prediction',
    metricName='accuracy'
)
accuracy = acc_eval.evaluate(eval_df)

# 2. Tính F1-Weighted (SỬA LẠI TÊN METRIC Ở ĐÂY)
# Các tên hợp lệ: 'f1', 'weightedFMeasure', 'weightedPrecision', 'weightedRecall'
f1_eval = MulticlassClassificationEvaluator(
    labelCol='label',
    predictionCol='prediction',
    metricName='weightedFMeasure' # <-- Đã đổi từ 'f1weighted' thành 'weightedFMeasure'
)
f1_score = f1_eval.evaluate(eval_df)

print("="*40)
print(f"TEST ACCURACY:       {accuracy:.4f}")
print(f"TEST F1-WEIGHTED:    {f1_score:.4f}")
print("="*40)

# cm = confusion_matrix(all_labels_flat, all_preds_flat)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
#             xticklabels=['Neg', 'Neu', 'Pos', 'None'], 
#             yticklabels=['Neg', 'Neu', 'Pos', 'None'])
# plt.xlabel('Dự đoán')
# plt.ylabel('Thực tế')
# plt.title('Confusion Matrix')
# plt.show()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.